In [0]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH = '/content/drive/My Drive/GU_neural_network/'

In [0]:
import sys
sys.path.append(PATH)

In [0]:
!pip install keras

## Практическое задание

<ol>
    2. Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения<br><br>

    
</ol>

##Константы

In [0]:
SEQLEN, STEP = 10, 2
RANDOM_STATE = 1
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 25 
NUM_EPOCHS_PER_ITERATION = 5
NUM_PREDS_PER_EPOCH = 100

## Библотеки

In [0]:
import numpy as np
import os
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential
from keras import regularizers

import warnings
warnings.filterwarnings(action='ignore', category=Warning)

Using TensorFlow backend.


In [0]:
np.random.seed(RANDOM_STATE)

## Подготовка данных

In [0]:
# построчное чтение из примера с текстом 
with open(os.path.join(PATH,"alice_in_wonderland.txt"), 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

In [0]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [0]:
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [0]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

## Дополнительные функции

## 1. Модель

In [0]:
def baseline_model(name='baseline', size=HIDDEN_SIZE, seq_lenght=SEQLEN, nb_chars=100):
  # Build the model.
  model = Sequential(name=name)
  model.add(
      GRU(  
          size,
          return_sequences=True,
          input_shape=(seq_lenght, nb_chars),
          unroll=True
      )
  )
  model.add(
      GRU(  
          size,
          return_sequences=False,
          input_shape=(seq_lenght, nb_chars),
          dropout=0.2, recurrent_dropout=0.5,
          unroll=True
      )
  )
  model.add(Dense(nb_chars))
  model.add(Activation("softmax"))
  model.compile(loss="categorical_crossentropy", optimizer="adam")
  model.summary()
  return model

## 2. Обучение

In [0]:
model = baseline_model(nb_chars=nb_chars)

# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Model: "baseline"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 10, 128)           64896     
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_1 (Dense)              (None, 40)                5160      
_________________________________________________________________
activation_1 (Activation)    (None, 40)                0         
Total params: 168,744
Trainable params: 168,744
Non-trainable params: 0
_________________________________________________________________
Итерация #: 0
Epoch 1/5
69762/69762 [==============================] - 8s 112us/step - loss: 2.5381
Epoch 2/5
69762/69762 [==============================] - 6s 91us/step - loss: 2.0450
Epoch 3/5
69762/69762 [==============================] - 7s 100us

##Вывод:
Текст получился более осмысленный благодаря:
1. Увеличению количества эпох
2. Исключениею из текста технической и юридической информации
3. Увеличение количества итераций
4. Увеличение количества нейронов в слое, но добавлние еще одного слоя показало лучшие результаты с сопоставимым количеством параметров

Ухудшило:  
1. LSTM с регуляризацией и без требовало долгого обучения, но при этом текст генерировался с постоянными повторениями
"voice, let up of the mock turtle the mock turtle the mock turtle the mock turtle the mock turtle the mock turtle".   
Наилуший вариант был: "she would get up and the repeatied of the mouse to the mouse she said to herself, and the mouse to the mouse sh"
2. Снижение количества нейронов в слое

Лучшее:
"out of the words dread of any more minute as the caterpillar was the pigeon to here, she was all the rest" - "из слов страха еще одной минуты, как гусеница была голубем здесь, она была всем остальным"  
"said, and the jury all was to many at and got but it mouse, that feet little course who was gont me near"  - "сказал, а присяжные все были ко многим приставлены и достали но ей мышь, что ноги мало ходила кто собирался меня рядом"